In [5]:
import numpy as np
import torch
from torch.distributions.normal import Normal as N

$CRPS(\mathcal{N}(\mu, \sigma), x) = \sigma \bigl[ \frac{1}{\sqrt{\pi}} - 2 \phi(\frac{x-\mu}{\sigma}) - \frac{x-\mu}{\sigma}\bigl(2\Phi(\frac{x-\mu}{\sigma}) -1 \bigr)\bigr]$

In [17]:
def crps_gaussian(mu, sigma, x):
    '''Function to compute the Continuous Ranked Probability Score (CRPS) of an observed value, using a Gaussian
    distribution with mean mu and variance sigma^2 as a reference. 
    
    Parameters
    ----------
    mu : float
        The mean of the reference Gaussian distribution
    sigma : float
        The standard deviation of the reference Gaussian distribution
    x : float
        The observed value to score
    
    Returns
    -------
    crps : torch.tensor
        The CRPS of the observed value
    '''
    
    # Convert float values to tensors for use in PyTorch functions
    mu, sigma, x = torch.tensor([mu]), torch.tensor([sigma]), torch.tensor([x])
    
    # Compute shorthands of the normalized observation and pi, and create a standard normal distribution
    norm = (x-mu) / sigma
    Z = N(loc=0, scale=1)
    pi = torch.acos(torch.zeros(1)).item() * 2
    
    # Compute CRPS
    crps = sigma * (1/torch.sqrt(torch.tensor(pi)) - 2*torch.exp(Z.log_prob(norm)) - norm*(2*Z.cdf(norm) -1))
    return crps 

In [18]:
crps_gaussian(torch.tensor([0.1]), torch.tensor([0.01]), 1)

tensor([-0.8944])

In [19]:
crps_gaussian(torch.tensor([0.1]), torch.tensor([0.01]), 0.1)

tensor([-0.0023])

In [ ]:
# TODO: test against pip CRPS package, use ensemble of Gaussian observations